In [28]:
from enum import Enum
class Keywords(Enum):
    # 검사대상
    product_name = '제품명'
    weight = '중량(수량)'
    barcode = '바코드'
    cert_mark = '인증마크'
    cert_result = '인증정보'

    # 검사 / pass
    inspect = '검사'
    pass_inspect = 'pass'

    # 인증마크
    organic = '유기농'
    nonpesticide = '무농약'
    gap = 'GAP(우수관리인증)'
    antibiotic = '무항생제'
    animal = '동물복지'
    haccp = '안전관리인증HACCP'
    pgi = '지리적표시'
    traditional = '전통식품'
    master = '식품명인'
    processed = '가공식품'
    carbon = '저탄소(LOW CARBON)'

    # 연속 / 누적 검사 결과
    success_cs = '합격(연속)'
    success_cu = '합격(누적)'
    fail_cs = '불합격(연속)'
    fail_cu = '불합격(누적)'
    pass_cs = '유보(연속)'
    pass_cu = '유보(누적)'
    total = '총'

    def __str__(self):
        return self.name

    def kor(self):
        return self.value

    def eng(self):
        return self.name

    def insp_headers(self):
        return [
            self.product_name,
            self.weight,
            self.barcode,
            self.cert_mark,
            self.cert_result
        ]

    def error_headers(self):
        return [
            self.product_name,
            self.success_cs,
            self.fail_cs,
            self.pass_cs,
            self.success_cu,
            self.fail_cu,
            self.pass_cu,
            self.total
        ]

In [30]:
for item in list(Keywords.error_headers(Keywords)):
    if '_cs' in str(item):
        print(str(item))

success_cs
fail_cs
pass_cs


In [31]:
class PdCumul:
    def __init__(self, pd_name):
        self.pd_name = pd_name
        self.final_result_txt = {
            '합격': Keywords.success_cs,
            '유보': Keywords.pass_cs,
            '불합격': Keywords.fail_cs
        }
        self.cu_dict = {}
        self.cs_dict = {}
        self.prior_target = ''
        error_headers = Keywords.error_headers(Keywords)
        for item in list(error_headers):
            if '_cs' in str(item):
                self.cs_dict[str(item)] = 0
            else:
                self.cu_dict[str(item)] = 0

    def clear(self):
        for key in self.cs_dict.keys():
            self.cs_dict[key] = 0

        for key in self.cu_dict.keys():
            self.cu_dict[key] = 0

    def add(self, target):
        target = str(target)
        self.cs_dict[target] += 1
        if self.prior_target != target:
            self.prior_target = target
            rest = [item for item in self.cs_dict.keys() if item != target]
            for item in rest:
                self.cs_dict[item] = 0
        cumul_target = target.replace('_cs', '_cu')
        self.cu_dict[cumul_target] += 1
        self.cu_dict['total'] += 1

    def rslt_dict(self):
        result = {}
        result.update(self.cs_dict)
        result.update(self.cu_dict)
        return result

    def product_name(self):
        return self.pd_name

In [36]:
pd = PdCumul('유기농 방울토마토')
pd.add(Keywords.success_cs)
pd.rslt_dict()

{'success_cs': 1,
 'fail_cs': 0,
 'pass_cs': 0,
 'product_name': 0,
 'success_cu': 1,
 'fail_cu': 0,
 'pass_cu': 0,
 'total': 1}